In [1]:
# check python version
import sys
print(sys.version)

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]


In [2]:
# check if transformers and tensorflow are installed, if not install them
# use transformers version 4.18.0 and tensorflow version 2.8.0
try:
    import transformers
    import tensorflow as tf
    print("transformers and tensorflow are installed")
except:
    print("transformers and tensorflow are not installed")
    print("installing transformers and tensorflow")

d:\DH\projects\Bangkit-C22CB-Company-Based-Capstone\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


transformers and tensorflow are installed


In [31]:
model_name = "cahya/bert-base-indonesian-522M"
batch_size = 32

from transformers import BertTokenizer, TFAutoModel # make sure use tensorflow model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name)

Downloading: 100%|██████████| 224k/224k [00:03<00:00, 68.3kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<00:00, 55.9kB/s]
Downloading: 100%|██████████| 62.0/62.0 [00:00<00:00, 62.0kB/s]
Downloading: 100%|██████████| 468/468 [00:00<00:00, 156kB/s]
Downloading: 100%|██████████| 520M/520M [04:27<00:00, 2.03MB/s]   
Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-

In [32]:
model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
Total params: 110,617,344
Trainable params: 110,617,344
Non-trainable params: 0
_________________________________________________________________


In [42]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizer)

In [43]:
# test tokenizer
tokenizer("Nama kamu siapa?")

{'input_ids': [3, 1769, 8343, 6186, 32, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [49]:
tokenizer("saya suka makan nasi goreng")

{'input_ids': [3, 3245, 5366, 2464, 6014, 11186, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [50]:
unmasker = transformers.pipeline('fill-mask', model = model_name)
unmasker("mainan saya [MASK] di jalan")

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


[{'score': 0.0840364545583725,
  'token': 2186,
  'token_str': 'berada',
  'sequence': 'mainan saya berada di jalan'},
 {'score': 0.07038316130638123,
  'token': 1821,
  'token_str': 'ada',
  'sequence': 'mainan saya ada di jalan'},
 {'score': 0.0403575636446476,
  'token': 1998,
  'token_str': 'sendiri',
  'sequence': 'mainan saya sendiri di jalan'},
 {'score': 0.029048316180706024,
  'token': 2444,
  'token_str': 'lahir',
  'sequence': 'mainan saya lahir di jalan'},
 {'score': 0.028137197718024254,
  'token': 3812,
  'token_str': 'berdiri',
  'sequence': 'mainan saya berdiri di jalan'}]

In [46]:
%pip install pandas

In [47]:
# load dataset json file
import json

dev_json_dir = "Translated/dev-v2.0_indo.json"

# open json
with open(dev_json_dir, encoding="utf-8") as json_file:
    file = json.load(json_file)
    dict_file = file
    data = dict_file['data']

In [48]:
'''
    This function is for converting SQuAD json file to pandas dataframe
'''

import utils

# call function

#dev_df = utils.json_to_df(data)
#dev_df.head()

# save dataframe to csv
#dev_df.to_csv('TranslatorSQuAD/Translated/dev-v2.0_indo.csv', index=False)

# dataset detail
#dev_df.shape

---